# 09. CODE SEARCH: TWO-STAGE PARADIGM

1. [x] введение
2. [x] TOSS
3. [x] evaluation
4. [x] результаты
5. [x] ссылки

# 1. Введение

Существующие подходы поиска по коду условно можно разделить на две группы:
1. методы на основе Information retrieval: они часто работают быстро, но иногда не совсем точны
2. методы на основе DL: могут показывать высокое качество, но обычно более медленные из-за использования больших моделей.

Среди DL-подходов, в свою очередь, можно выделить *биэнкодерные (bi-encoder)* и *кроссэнкодерные (cross-encoder)* решения.

## 1. Биэнкодерный подход

В биэнкодерном подходе входные данные (текстовый запрос, код) независимо вкладываются в векторное пространство.

![](./res/09_bi-encoder.png)

Таким образом, эмбеддинги для кода могут быть вычислены заранее, что ускоряет работу поиска.

## 2. Кроссэнкодерный подход

В отличие от бинэнкодеров в случае кроссэнкодера механизм эттеншна выполняется на паре запрос-код, таким образом  происходит взаимодействие информации на естественном языке с информацие на языке программирования в процессе векторизации.

![](./res/09_cross-encoder.png)

За счёт этого кроссэнкодерные подходы могут достичь более высокую точность, чем биэнкодерные решения.
Но кроссэнкодерные подходы не могут создавать отдельные эмбеддинги для коды, необхожимые для многих приложений.

# 2. TOSS

Как объединить преимущества всех походов?

![](./res/09_toss_paper.png)

В работе [Revisiting Code Search in a Two-Stage Paradigm](https://arxiv.org/abs/2208.11274) авторы предложили
TOSS (TwO-Stage fuSion code Search framework) --- двухэтапный фреймворк для поиска по коду, который сочетает в себе преимущества различных методов поиска кода.
TOSS сначала использует модели на основе Information Retrieval и биэнкодерные модели для эффективного поиска небольшого количества лучших кандидатов, а затем использует кроссэнкодеры для более точного ранжирования.

## 2.1 Обозначения и вспомогательные понятия

Пусть
- $q$ --- поисковый запрос
- $C$ --- корпус кода (где происходит поиск)
- $M(\cdot, \cdot)$ --- функция соответствия (match) сниппета кода $c$ поисковому запросу $q$, чем больше значение фунции $M$, тем лучше $c$ соответствует $q$.

Тогда поиск по коду --- это поиск такого $c$ для запроса $q$, что значение $M(c, q)$ максимально:
$$max_{c \in C}M(c, q).$$

#### Биэнкодерный подход

Пусть $c \in C$ --- сниппет кода из корпуса $C$.
Пусть $\Gamma'$ и $\Gamma''$ --- энкодеры для запросов и сниппетов кода, соответственно.
Тогда через $e_q$ и $e_c$ обозначим эмбеддинги для $q$ и $c$:
$$e_q = \Gamma'(q)$$
$$e_c = \Gamma''(c).$$
Кроме того, нам необходима некоторая функция похожести или функция расстояния $s_{bi}$ для векторов.
Например, косинус угла между векторами или евклидово расстояние:
$$s_{bi}: (e_q, e_c) \mapsto R.$$

Таким образом, имеем $$M_{bi}(\cdot, \cdot) = s_{bi}(\Gamma'(\cdot), \Gamma''(\cdot)).$$

Энкодеры $\Gamma'$ и $\Gamma''$ --- нейронные сети, предварительно обученные на парах (текст, код).

#### Кроссэнкодерный подход

В этом подхоже поисковый запрос и сниппет кода объединяются (конкатенируются через специальный токен $<SEP>$) и являются входом для модели $\Gamma$, которая вычисляет кроссмодальную похожесть (энкодер+регрессия):

$$M_{cr}(\cdot, \cdot) = \Gamma([q, <SEP>, c]).$$

## 2.2 Двухэтапный поиск по коду

1. Сначала находим набор сниппетов-кандидатов на основе функции $M_{recall}$.
2. Затем ранжируем набор кандидатов на основе функции $M_{rank}$.

![](./res/09_toss_overview.png)

Если одна модель на первом этапе.

1. $C_{sub} = \arg \max_{C' \subset C, |C'| = K} \sum_{c \in C'} M_{recall}(c, q)$ --- набор из $K$ кандидатов.

Функция $M_{recall}$ должна быть быстрой, но, возможно, неточной.

2. $c_{*}   = \arg \max_{c \in C_{sub}}                          M_{rank  }(c, q)$ --- итоговый сниппет.

Функци я $M_{rank}$ может быть сложной.


Кроме того, первый этап ($M_{recall}$) может включать несколько $m$ различных способов поиска по коду. Полученные кандидаты объединяются и отправляются на второй этап:

$$C^{i}_{sub} = \arg \max_{C' \subset C, |C'| = K} \sum_{c \in C'} M^i_{recall}(c, q), \,\,\, 1 \leq i \leq m$$
$$C_{all} = \cup_{1 \leq i \leq m}C^i_{sub}$$

# 3. Evaluation

Как оценить качество?

## 3.1 Датасет

- [CodeSearchNet](https://github.com/github/CodeSearchNet) --- 14919 запросов и 43827 кандидатов

## 3.2 Модели

Требования к выбору моделей:
- исходный код опубликован
- модели работают со всеми шестью языками программирования из CodeSearchNet
- исследовательская статья прошла рецензирование

В результате следующие модели были выбраны:
- на основе Information retrieval
  - [Jaccard](https://www.scienceopen.com/document?vid=01a9bb0e-a14c-461f-85c9-f407259515dc): был использован метод `jaccard_score` из библиотеки `sklearn`
  - [BOW](https://nlp.stanford.edu/IR-book/information-retrieval-book.html): был использовае `CountVectorizer` для векторизации из библиотеки `sklearn`, кроссмодальная похожесть оценивалась через косинус угла между векторами
  - [TFIDF](https://asistdl.onlinelibrary.wiley.com/doi/abs/10.1002/asi.4630270302): был использован `TfidfVectorizer` из библиотеки `sklearn` на основе TFIDF-признаков, кроссмодальная похожесть оценивалась через косинус угла между векторами
  - [BM25](https://www.nowpublishers.com/article/Details/INR-019): был использован Python-пакет `Rank-BM25`
- биэнкодерные
  - [CODEnn](https://github.com/guxd/deep-code-search): на основе LSTM
  - [GraphCodeBERT](https://github.com/microsoft/CodeBERT/tree/master/GraphCodeBERT/codesearch)
  - [CodeBERT-bi](https://arxiv.org/abs/2002.08155): вариант CodeBERT, описан в Приложении; модель сначала кодирует запрос и код раздельно, а затем вычисляет сходство через скалярное произведение
- кроссэнкодерные
  - [CodeBERT](https://github.com/microsoft/CodeBERT)
  - [CoCLR](https://github.com/Jun-jie-Huang/CoCLR): метод constrastive learning на основе CodeBERT
  
  
 ## 3.3 Метрики
 
 ### MRR
 
MRR (Mean Reciprocal Ranking) --- статистическая оценка откликов процесса на запросы, упорядоченных по вероятности и правильности. Характеризует эффективность информационного поиска.
MRR определяется как среднее обратных рангов по всем запросам $q$ из $Q$:

$$MRR = \frac{1}{|Q|} \sum_{q \in Q} \frac{1}{rank(q)},$$

где $rank(q)$ означает положение первого релевантного документа для запроса $q$.
 
 ### R@K
 
R@K (K=1, 5, 10, 100, 1000) --- количество раз, когда корректный ответ попал в топ-K ответов на запрос.

# 4. Результаты

## 4.1 Предобработка

![](./res/09_preprocessing.png)

- SPS: разбить слова согласно написанию в стиле pascal или snake case, например, `TwoStageMethod` преваращется в  `Two Stage Method”`
- DS: удалить английские стоп-слова, например, `both`, `more`, `some` и т.д.
- RS: Ronin Split, например, преобразовать `showtraceback` в `show trace back`
- POS: Part-of-speech restoration, лемматицзация, например, `configs` в `config`

## 4.2 Baselines

BM25 --- поисковая функция на неупорядоченном множестве термов ("мешке слов") и множестве документов, которые она оценивает на основе встречаемости слов запроса в каждом документе, без учёта взаимоотношений между ними (например, близости).

![](./res/09_baselines.png)

- DL-подходы дают более высокое качество
- лучше всех кроссэнкодерный подход на основе CodeBERT
- кросс-энкодерные подходы медленные

![](./res/09_baselines_viz.png)

## 4.1 Качество

![](./res/09_toss_results.png)

- TOSS улучшает качество
- Подход [GraphCodeBERT + BM25] + CodeBERT работает лучше всех

![](./res/09_toss_langs.png)

## 4.2 Скорость

![](./res/09_toss_perfomance.png)
![](./res/09_toss_speed.png)

# 5. Полезные ссылки

- Birillo et al - Reflekt A library for compile-time reflection in Kotlin 2022
- Chai et al - Cross-domain deep code search with few-shot meta learning 2022
- Cheng Kuang - CSRS Code search with relevance matching and semantic matching 2022
- Eberhart McMillan - Generating clarifying questions for query refinement in source code search 2022
- Grazia et al - DiffSearch A scalable and precise search engine for code changes 2022
- Gu et al - Accelerating code search with deep hashing and code classification 2022
- Gu et al - Multimodal representation for neural code search 2022
- Haldar et al - A multi-perspective architecture for semantic code search 2021
- [Hu et al - Revisiting code search in a two-stage paradigm 2022](https://arxiv.org/abs/2208.11274)
- Li et al - CodeRetriever Unimodal and bimodal contrastive learning for code search 2022
- Liu et al - CodeMatcher Searching code based on sequential semantics of important query words 2022
- Liu et al - GraphSearchNet Enhancing GNNs via capturing global dependency for semantic code search 2022
- Shi et al - Enhancing semantic code search with multimodal contrastive learning and soft data augmentation 2022
- Shuai et al - Improving code search with co-attentive representation learning 2020
- Sun et al - Code search based on context-aware code translation 2022
- Sun et al - On the importance of building high-quality training datasets for neural code search 2022
- Villmow et al - Addressing leakage in self-supervised contextualized code retrieval 2022
- Wang et al - Enriching query semantics for code search with reinforcement learning 2021
- Yan et al - Are the code snippets what we are searching for A benchmark and an empirical study on code search with natural-language queries 2020
- Zhang et al - Bag-of-words baselines for semantic code search 2021
- https://ieeexplore.ieee.org/abstract/document/8453172
- https://dl.acm.org/doi/abs/10.1145/2393596.2393606
- https://ieeexplore.ieee.org/abstract/document/6606630